In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Exercise 2:

### Melbourne Pedestrians

Imagine you are asked to put together an analysis of Pedestrian trends for an OOH campaign

- This exercise will concentrate on showing Pandas key features

- This example uses data from the City of Melbourne pedestrian counters, found at http://www.pedestrian.melbourne.vic.gov.au/.

### Part 1: What are DataFrames


In [ ]:
# Import pandas
import pandas as pd

# Create sample data
index = ['2021-07-01', '2021-07-02','2021-07-03', '2021-07-04']
data ={"day":[1, 2, 3, 4],"month":[7,7,7,7],"year":[2021, 2021, 2021, 2021]}

# Generate sample DataFrame
df = pd.DataFrame(index=index, data=data)
df

#### DataFrames as "collection" of columns


In [ ]:
# select 'one' column
df[["day"]]

In [ ]:
# select 'many' column
df[["day", "month"]]

In [ ]:
# average or sum over columns
df.mean()

In [ ]:
# get unique values in columns
pd.unique(df["month"])

In [ ]:
# concatenate or operate on colums

df["date_str"] = df.year.astype(str) + '-' + df.month.astype(str) + '-' + df.day.astype(str)
df

In [ ]:
# apply function to column

def date_string_to_datetime(column):
    return pd.to_datetime(column)


df["date_datetime"] = df["date_str"].apply(date_string_to_datetime)
df

In [ ]:
df.date_datetime.dt.month



The below looks at:
* Loading the daily counts from a file
* Transforming the data to time series layout
* Merging data from different files together
* Grouping/aggregating data

### Part 2: Pedestrian data

By inspecting the CSV files in Excel we can identify the rows containing the daily pedestrian counts and load only that section.

In [ ]:
import pandas as pd

raw_df = pd.read_csv("data/2_pedestrians/COM_24PM_21-06-2021.csv", skiprows=8, nrows=65)
raw_df.head()

#### Transpose

Getting things into the right format for this data is pretty straightforward.
We first need to set the `Sensor` column as the index.

In [ ]:
df = raw_df.set_index("Sensor")
df.head()

`df.transpose()` then transposes the data, swapping rows with columns.

In [ ]:
transposed = df.transpose()
transposed.head()

This is not bad, we have rows for observations (hourly) and columns for measurements (different locations).

We can use this to plot different measurements over time.
Pandas will plot one line series for each column measurement and align them over time.

In [ ]:
transposed[["Monash Rd-Swanston St (West)","Bourke St - Spencer St (North)", "RMIT Building 14"]]\
    .plot.line(figsize=(15, 6))


This is ok but it's not being treated as a time series yet.
We need to replace the index with a time-based instead of text-based one.
`pd.date_range` does what we need, giving hourly labels from midnight to 11pm.

In [ ]:
pd.date_range(start="2021-06-21", periods=24, freq='H')

This step replaces the string-labelled index with the time-based one.
Note that this relies on the index already being in the correct order, and the new index must be the same length as the old one.

In [ ]:
timeseries_df = (
    transposed
    .assign(time_of_day=pd.date_range(start="2021-06-21", periods=24, freq='H'))
    .set_index("time_of_day")
)
timeseries_df.head()

In [ ]:
(
    timeseries_df[[
        "Monash Rd-Swanston St (West)",
        "Bourke St - Spencer St (North)",
        "RMIT Building 14",
    ]]
    .plot.line(figsize=(15, 6))
)

## Selecting Data

With data in this form we can select rows of the data using time-based ranges.

e.g. This selects a 4 hour period in the middle of the day ...

In [ ]:
# Select a 4 hour period
lunchtime_data = timeseries_df.loc[pd.Timestamp("2021-06-21 11:00:00"):pd.Timestamp("2021-06-21 14:00:00")]
lunchtime_data

... then extract a sum total over this period for each sensor and plot ...

In [ ]:
# For each column, sum over all the rows in the selected subset of data.
lunchtime_totals = lunchtime_data.sum(axis='index')

# Sort by total count.
lunchtime_totals = lunchtime_totals.sort_values()

# Plot the results as a bar chart.
lunchtime_totals.sort_values().plot.barh(figsize=(8, 15));

#### Concatenating Data

The pedestrian dataset is spread across different files for each day.
In order to collect it into one dataframe, we can use two steps.

1. Write a python function which reads data for one day and performs our read, transpose, and labelling steps.
2. Concatenate the different data sources with `pd.concat`.

A Python [function](https://www.w3schools.com/python/python_functions.asp) encapsulates a set of operations so they can be repeated for different inputs.
In this case, the function reads the data file for a given day, transposes the data, adds a timestamp index, and returns the processed dataframe.

In [ ]:
import os
def read_for_date(date):
    """ This function  """
    raw_df = pd.read_csv(os.path.join("data","2_pedestrians","COM_24PM_"+ date + ".csv"), skiprows=8, nrows=65)
    transposed = raw_df.set_index("Sensor").transpose()
    timestamped_df = (
        transposed
        .assign(Timestamp=pd.date_range(start=date, periods=24, freq='H'))
        .set_index("Timestamp")
    )
    return timestamped_df

# Concatenate daily data for a week.
df = pd.concat([
    read_for_date("20-06-2021"),
    read_for_date("21-06-2021"),
    read_for_date("22-06-2021"),
    read_for_date("23-06-2021"),
    read_for_date("24-06-2021"),
    read_for_date("25-06-2021"),
    read_for_date("26-06-2021"),
])

#### Loops

It's useful to know how to write a *loop* or *list comprehension* in Python to simplify repetitive tasks.
One simple use is the following, which selects all column names containing 'Bourke' from the columns of a dataframe.

In [ ]:
[column_name for column_name in df.columns if "Bourke" in column_name]

This can be used to select this list of columns instead of writing the names out manually.

In [ ]:
bourke_st_sensors = df[[column_name for column_name in df.columns if "Bourke" in column_name]]
bourke_st_sensors.plot.line(figsize=(15, 6));

#### Time-based Aggregation and Pivoting

We can use the timestamped index to add some labelling columns:
* The date (without time)
* The time of day (without date)
* The day of the week

In [ ]:
df = df.assign(
    Date=lambda df: df.index.floor("D"),
    TimeOfDay=lambda df: df.index - df.Date,
    DayOfWeek=lambda df: df.index.day_name().astype("category"),
)
df.head()

These labels can be used to aggregate and summarise data.
`groupby` collects data according to the values of some selected columns and calculates summary statistics on each group.
The result is a summary row for each group.

In [ ]:
aggregated = df.groupby(["DayOfWeek", "TimeOfDay"]).median()
aggregated

The `unstack` and `stack` functions move labels from the index to columns (and vice-versa).
In this case `unstack` expands the day of the week to produce multiple column levels.

In [ ]:
pivoted = aggregated.unstack(0)
pivoted.head()

Selecting a column now just refers to the first level in the column heirarchy.
As a result this selection picks one sensor and returns a column for each day, with rows representing the time of day.

In [ ]:
bourke_st = pivoted["Bourke St - Spencer St (North)"]
bourke_st.head()

The result is median hourly counts, aggregated for each day of the week (over a full year) at the same location.

In [ ]:
bourke_st.plot.line(figsize=(15, 6));